In [ ]:
# Import everything
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import doubleml as dml
 
 
 
from numpy.linalg import inv
from statsmodels.iolib.summary2 import summary_col
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from linearmodels import IV2SLS, IVLIML, IVGMM, IVGMMCUE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.base import clone
from doubleml.datasets import make_pliv_CHS2015

# Preliminaries

In [ ]:
!pip install linearmodels
!pip install -U DoubleML

Requirement already up-to-date: DoubleML in /usr/local/lib/python3.6/dist-packages (0.1.2)


In [ ]:
# Import matplotlib for graphs
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Set global parameters
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['figure.titlesize'] = 20
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Import

In [ ]:
df = pd.read_stata('assignment2.dta')
df=df.reset_index(drop=False)
# Add constant term to dataset
df['const'] = 1

In [ ]:
df.head()

,index,X1,Z1,X2,Z2,y,dlhpop,X3,X4,X5,X6,X7,X8,X9,X10,X97,X108,X119,X130,X42,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,...,X138,X139,X140,X43,X44,X45,X46,X47,X21,X32,X36,X37,X38,X39,X40,X41,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X33,X34,X35,const
0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Columns: 206 entries, week to const
dtypes: float32(62), float64(3), int64(1), int8(140)
memory usage: 1.3 MB


In [ ]:
# Define a new variable containing all the week dummies
weekdummies = df.iloc[:,70:173];
weekdummies['_Iweek_1']=0
week=df['week'].values
for i in range(weekdummies.shape[0]):
    if week[i]==1.0:
        weekdummies.iloc[i,103]=1
weekdummies

,_Iweek_2,_Iweek_3,_Iweek_4,_Iweek_5,_Iweek_6,_Iweek_7,_Iweek_8,_Iweek_9,_Iweek_10,_Iweek_11,_Iweek_12,_Iweek_13,_Iweek_14,_Iweek_15,_Iweek_16,_Iweek_17,_Iweek_18,_Iweek_19,_Iweek_20,_Iweek_21,_Iweek_22,_Iweek_23,_Iweek_24,_Iweek_25,_Iweek_26,_Iweek_27,_Iweek_28,_Iweek_29,_Iweek_30,_Iweek_31,_Iweek_32,_Iweek_33,_Iweek_34,_Iweek_35,_Iweek_36,_Iweek_37,_Iweek_38,_Iweek_39,_Iweek_40,_Iweek_41,...,_Iweek_66,_Iweek_67,_Iweek_68,_Iweek_69,_Iweek_70,_Iweek_71,_Iweek_72,_Iweek_73,_Iweek_74,_Iweek_75,_Iweek_76,_Iweek_77,_Iweek_78,_Iweek_79,_Iweek_80,_Iweek_81,_Iweek_82,_Iweek_83,_Iweek_84,_Iweek_85,_Iweek_86,_Iweek_87,_Iweek_88,_Iweek_89,_Iweek_90,_Iweek_91,_Iweek_92,_Iweek_93,_Iweek_94,_Iweek_95,_Iweek_96,_Iweek_97,_Iweek_98,_Iweek_99,_Iweek_100,_Iweek_101,_Iweek_102,_Iweek_103,_Iweek_104,_Iweek_1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Define a new variable containing all the borough dummies
boroughdummies = df.iloc[:,173:204];
boroughdummies['_Iocu_1']=0
boroughdummies.iloc[:104,31]=1
boroughdummies

,_Iocu_2,_Iocu_3,_Iocu_4,_Iocu_5,_Iocu_6,_Iocu_7,_Iocu_8,_Iocu_9,_Iocu_10,_Iocu_11,_Iocu_12,_Iocu_13,_Iocu_14,_Iocu_15,_Iocu_16,_Iocu_17,_Iocu_18,_Iocu_19,_Iocu_20,_Iocu_21,_Iocu_22,_Iocu_23,_Iocu_24,_Iocu_25,_Iocu_26,_Iocu_27,_Iocu_28,_Iocu_29,_Iocu_30,_Iocu_31,_Iocu_32,_Iocu_1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Create lists of variables to be used in each regression
X1 = ['const', 'dlun', 'dlemp', 'dlymale', 'dlwhite','policy_treat1', 'post_treat1']
y = df['dlhpop']

In [ ]:
controls1 = weekdummies.join(df[X1])
controls = boroughdummies.join(controls1)
controls

,_Iocu_2,_Iocu_3,_Iocu_4,_Iocu_5,_Iocu_6,_Iocu_7,_Iocu_8,_Iocu_9,_Iocu_10,_Iocu_11,_Iocu_12,_Iocu_13,_Iocu_14,_Iocu_15,_Iocu_16,_Iocu_17,_Iocu_18,_Iocu_19,_Iocu_20,_Iocu_21,_Iocu_22,_Iocu_23,_Iocu_24,_Iocu_25,_Iocu_26,_Iocu_27,_Iocu_28,_Iocu_29,_Iocu_30,_Iocu_31,_Iocu_32,_Iocu_1,_Iweek_2,_Iweek_3,_Iweek_4,_Iweek_5,_Iweek_6,_Iweek_7,_Iweek_8,_Iweek_9,...,_Iweek_73,_Iweek_74,_Iweek_75,_Iweek_76,_Iweek_77,_Iweek_78,_Iweek_79,_Iweek_80,_Iweek_81,_Iweek_82,_Iweek_83,_Iweek_84,_Iweek_85,_Iweek_86,_Iweek_87,_Iweek_88,_Iweek_89,_Iweek_90,_Iweek_91,_Iweek_92,_Iweek_93,_Iweek_94,_Iweek_95,_Iweek_96,_Iweek_97,_Iweek_98,_Iweek_99,_Iweek_100,_Iweek_101,_Iweek_102,_Iweek_103,_Iweek_104,_Iweek_1,const,dlun,dlemp,dlymale,dlwhite,policy_treat1,post_treat1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-0.096514,0.057115,0.113439,-0.016721,0.0,0.0
3324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.460686,-0.049120,0.252547,-0.079729,0.0,0.0
3325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-0.166312,0.013269,0.084387,0.059519,0.0,0.0
3326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-0.198136,0.062120,0.458232,-0.021839,0.0,0.0


In [ ]:
# Replicating Column 2, Table 2 Crime paper (without weights and clustered standard errors)
reg1 = sm.OLS(y, controls, missing='drop')
results = reg1.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 dlhpop   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     60.71
Date:                Tue, 12 Jan 2021   Prob (F-statistic):               0.00
Time:                        21:16:30   Log-Likelihood:                 2734.9
No. Observations:                1664   AIC:                            -5292.
Df Residuals:                    1575   BIC:                            -4810.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
_Iocu_2          -0.0284      0.007     -3.994

In [ ]:
#First stage regression
# Create lists of variables to be controlled for in first stage regression
X2 = ['const', 'policy', 'post', 'dlun', 'dlemp','dlymale', 'dlwhite','policy_treat1', 'post_treat1']
controls_fs = weekdummies.join(df[X2])
controls = boroughdummies.join(controls_fs)
controls

,_Iocu_2,_Iocu_3,_Iocu_4,_Iocu_5,_Iocu_6,_Iocu_7,_Iocu_8,_Iocu_9,_Iocu_10,_Iocu_11,_Iocu_12,_Iocu_13,_Iocu_14,_Iocu_15,_Iocu_16,_Iocu_17,_Iocu_18,_Iocu_19,_Iocu_20,_Iocu_21,_Iocu_22,_Iocu_23,_Iocu_24,_Iocu_25,_Iocu_26,_Iocu_27,_Iocu_28,_Iocu_29,_Iocu_30,_Iocu_31,_Iocu_32,_Iocu_1,_Iweek_2,_Iweek_3,_Iweek_4,_Iweek_5,_Iweek_6,_Iweek_7,_Iweek_8,_Iweek_9,...,_Iweek_75,_Iweek_76,_Iweek_77,_Iweek_78,_Iweek_79,_Iweek_80,_Iweek_81,_Iweek_82,_Iweek_83,_Iweek_84,_Iweek_85,_Iweek_86,_Iweek_87,_Iweek_88,_Iweek_89,_Iweek_90,_Iweek_91,_Iweek_92,_Iweek_93,_Iweek_94,_Iweek_95,_Iweek_96,_Iweek_97,_Iweek_98,_Iweek_99,_Iweek_100,_Iweek_101,_Iweek_102,_Iweek_103,_Iweek_104,_Iweek_1,const,policy,post,dlun,dlemp,dlymale,dlwhite,policy_treat1,post_treat1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.0,1.0,-0.096514,0.057115,0.113439,-0.016721,0.0,0.0
3324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.0,1.0,0.460686,-0.049120,0.252547,-0.079729,0.0,0.0
3325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0,1.0,-0.166312,0.013269,0.084387,0.059519,0.0,0.0
3326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,1.0,-0.198136,0.062120,0.458232,-0.021839,0.0,0.0


In [ ]:
df_for_reg_fs=controls.join(df['dlhpop'])
df_for_reg_fs=df_for_reg_fs.join(df['dltotpop'])
df_for_reg_fs=df_for_reg_fs.dropna(axis=0)
df_for_reg_fs
controls_fs=df_for_reg_fs.iloc[:,:145]

In [ ]:
controls_fs

,_Iocu_2,_Iocu_3,_Iocu_4,_Iocu_5,_Iocu_6,_Iocu_7,_Iocu_8,_Iocu_9,_Iocu_10,_Iocu_11,_Iocu_12,_Iocu_13,_Iocu_14,_Iocu_15,_Iocu_16,_Iocu_17,_Iocu_18,_Iocu_19,_Iocu_20,_Iocu_21,_Iocu_22,_Iocu_23,_Iocu_24,_Iocu_25,_Iocu_26,_Iocu_27,_Iocu_28,_Iocu_29,_Iocu_30,_Iocu_31,_Iocu_32,_Iocu_1,_Iweek_2,_Iweek_3,_Iweek_4,_Iweek_5,_Iweek_6,_Iweek_7,_Iweek_8,_Iweek_9,...,_Iweek_75,_Iweek_76,_Iweek_77,_Iweek_78,_Iweek_79,_Iweek_80,_Iweek_81,_Iweek_82,_Iweek_83,_Iweek_84,_Iweek_85,_Iweek_86,_Iweek_87,_Iweek_88,_Iweek_89,_Iweek_90,_Iweek_91,_Iweek_92,_Iweek_93,_Iweek_94,_Iweek_95,_Iweek_96,_Iweek_97,_Iweek_98,_Iweek_99,_Iweek_100,_Iweek_101,_Iweek_102,_Iweek_103,_Iweek_104,_Iweek_1,const,policy,post,dlun,dlemp,dlymale,dlwhite,policy_treat1,post_treat1
52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,-0.607333,-0.038377,-0.128403,-0.016142,0.0,0.0
53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.013031,-0.028857,-0.075578,-0.000569,0.0,0.0
54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,-0.420215,-0.054533,0.410244,0.033286,0.0,0.0
55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,-0.379971,-0.071872,-0.076856,-0.076864,0.0,0.0
56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,-0.170277,0.190105,0.216769,0.178514,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.0,1.0,-0.096514,0.057115,0.113439,-0.016721,0.0,0.0
3324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.0,1.0,0.460686,-0.049120,0.252547,-0.079729,0.0,0.0
3325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0,1.0,-0.166312,0.013269,0.084387,0.059519,0.0,0.0
3326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,1.0,-0.198136,0.062120,0.458232,-0.021839,0.0,0.0


In [ ]:
#First stage regression
reg_fs = sm.OLS(df_for_reg_fs['dlhpop'], controls_fs, missing='drop')
results_fs = reg_fs.fit()
print(results_fs.summary())

                            OLS Regression Results                            
Dep. Variable:                 dlhpop   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     60.71
Date:                Tue, 12 Jan 2021   Prob (F-statistic):               0.00
Time:                        21:16:40   Log-Likelihood:                 2734.9
No. Observations:                1664   AIC:                            -5292.
Df Residuals:                    1575   BIC:                            -4810.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
_Iocu_2          -0.0286      0.007     -4.030

In [ ]:
controls_secondstage=df_for_reg_fs.iloc[:,:143]
controls_secondstage['firststage_dlhpop'] = results_fs.predict()

In [ ]:
# Second stage
results_ss = sm.OLS(df_for_reg_fs['dltotpop'], controls_secondstage ,missing='drop').fit()

# Print3
print(results_ss.summary())

                            OLS Regression Results                            
Dep. Variable:               dltotpop   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     7.347
Date:                Tue, 12 Jan 2021   Prob (F-statistic):           1.78e-68
Time:                        21:16:45   Log-Likelihood:                 1509.5
No. Observations:                1664   AIC:                            -2843.
Df Residuals:                    1576   BIC:                            -2366.
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
_Iocu_2              -0.0491      0.01

# Double ML

In [ ]:
# 1. Split the sample in two: auxiliary sample (train) and main sample (test)
train, test = train_test_split(df_for_reg_fs, train_size=0.5, random_state=42)

df_aux = pd.DataFrame(train, columns=df_for_reg_fs.columns)
df_main = pd.DataFrame(test, columns=df_for_reg_fs.columns)


In [ ]:
# 2. Generate variables
y1 = df_aux['dltotpop'].values.reshape(-1,1)
D1 = df_aux[['dlhpop']].values.reshape(-1,1)
X1 = df_aux.iloc[:,:143]
Z1 = df_aux.iloc[:,:145]

y2 = df_main['dltotpop'].values.reshape(-1,1)
D2 = df_main[['dlhpop']].values.reshape(-1,1)
X2 = df_main.iloc[:,:143]
Z2 = df_main.iloc[:,:145]


In [ ]:
# 3. Residualize y1 on D1
reg1 = LinearRegression()
reg1.fit(D1, y1)
y_resid1 = y1-(reg1.predict(D1))

In [ ]:
# 4. Estimate g0
tree1 = RandomForestRegressor(max_leaf_nodes=5, random_state=42)
tree1.fit(X=X1, y=y_resid1.ravel())
g0 = tree1.predict(X2).reshape(-1,1)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=5,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
# 5. Compute u_hat
u_hat = y2 - g0

In [ ]:
# 6. Estimate m0
tree2 = RandomForestRegressor(max_leaf_nodes=5, random_state=42)
tree2.fit(X=Z1, y=D1.ravel())
m0 = tree2.predict(Z2).reshape(-1,1)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=5,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
# 7. Compute v_hat
v_hat = D2 - m0

In [ ]:
# 8. Estimate beta
beta = inv(v_hat.T @ D1) @ v_hat.T @ u_hat
beta

array([[3.8971006]])

Using Matteo's Code:

In [ ]:
# Generate variables
D = df_for_reg_fs[['dlhpop']].values.reshape(-1,1)
X = df_for_reg_fs.iloc[:,:143]
y = df_for_reg_fs['dltotpop'].values.reshape(-1,1)
Z = df_for_reg_fs.iloc[:,:145]

In [ ]:
def estimate_beta(algorithm, alg_name, D, X, y, Z, sample):

    # Split sample
    D_main, D_aux = (D[sample==1], D[sample==0])
    X_main, X_aux = (X[sample==1], X[sample==0])
    y_main, y_aux = (y[sample==1], y[sample==0])
    Z_main, Z_aux = (Z[sample==1], Z[sample==0])

    # Residualize y on D
    b_hat = inv(D_aux.T @ D_aux) @ D_aux.T @ y_aux
    y_resid_aux = y_aux - D_aux @ b_hat
    
    # Estimate g0
    alg_fitted = algorithm.fit(X=X_aux, y=y_resid_aux.ravel())
    g0 = alg_fitted.predict(X_main).reshape(-1,1)

    # Compute v_hat
    u_hat = y_main - g0

    # Estimate m0
    alg_fitted = algorithm.fit(X=Z_aux, y=D_aux.ravel())
    m0 = algorithm.predict(Z_main).reshape(-1,1)
    
    # Compute u_hat
    v_hat = D_main - m0

    # Estimate beta
    beta = inv(v_hat.T @ D_main) @ v_hat.T @ u_hat
        
    return beta 

In [ ]:
def ddml(algorithm, alg_name, D, X, y, Z, p=0.5, verbose=False):
    
    # Expand X if Lasso or Ridge
    if alg_name in ['Lasso   ','Ridge   ']:
        X = PolynomialFeatures(degree=2).fit_transform(X)

    # Generate split (fixed proportions)
    split = np.array([i in train_test_split(range(len(D)), test_size=p)[0] for i in range(len(D))])
    
    # Compute beta
    beta = [estimate_beta(algorithm, alg_name, D, X, y, Z, split==k) for k in range(2)]
    beta = np.mean(beta)
     
    # Print and return
    if verbose:
        print('%s : %.4f' % (alg_name, beta))
    return beta

In [ ]:
# List all algorithms
algorithms = {'Ridge   ': Ridge(alpha=.1),
              'Lasso   ': Lasso(alpha=.01),
              'Tree    ': DecisionTreeRegressor(),
              'Forest  ': RandomForestRegressor(n_estimators=30),
              'Boosting': GradientBoostingRegressor(n_estimators=30)}

In [ ]:
# Loop over algorithms
for alg_name, algorithm in algorithms.items():
    ddml(algorithm, alg_name, D, X, y, Z, verbose=True)

Ridge    : 0.0366
Lasso    : 0.1446
Tree     : 0.1697
Forest   : 0.0969
Boosting : 0.0997


In [ ]:
# Repeat K times
def estimate_beta_median(algorithms, D, X, y, Z, K):
    
    # Loop over algorithms
    for alg_name, algorithm in algorithms.items():
        betas = []
            
        # Iterate n times
        for k in range(K):
            beta = ddml(algorithm, alg_name, D, X, y, Z)
            betas = np.append(betas, beta)
    
        print('%s : %.4f' % (alg_name, np.median(betas)))

In [ ]:
np.random.seed(123)

# Repeat 100 times and take median
estimate_beta_median(algorithms, D, X, y, Z, 500)

Ridge    : 0.0518
Lasso    : 0.1437
Tree     : 0.0971
Forest   : 0.0782
Boosting : 0.1102


Using Double - ML Package: https://docs.doubleml.org/stable/guide/models.html#partially-linear-iv-regression-model-pliv 

In [ ]:
learner = RandomForestRegressor(n_estimators=100, max_features=20, max_depth=5, min_samples_leaf=2)
ml_g = clone(learner)
ml_m = clone(learner)
ml_r = clone(learner)
np.random.seed(2222)
data = df
obj_dml_data = dml.DoubleMLData(data, 'y', 'd', z_cols='Z1')
dml_pliv_obj = dml.DoubleMLPLIV(obj_dml_data, ml_g, ml_m, ml_r)
print(dml_pliv_obj.fit())